<a href="https://colab.research.google.com/github/sbbzplt/QiskitFallFest_IZTECH/blob/master/Grovers_maxcut_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from qiskit import *

### Task 1

Decide if the given graph has **3 cut-edges** where vertices V and edges E are:<br/>
V: (0,1,2,3)<br/>
E: (0,1),(0,2),(0,3)  
by using Grover's Algorithm with **iteration = 2** times.

Use qubits 0,1,2,3 for the vertices, qubits 4,5,6 to store the result of edge checks, 7 and 8 to store the count and 9 as output.

Complete the oracle function.

### Solution

In [ ]:
#cutedge checker
#Creates a circuit with n qubits, checks edge ab, and stores the result in c
def ccheck(n, a, b, c):
    qc = QuantumCircuit(n)
    qc.cx(a,c) 
    qc.cx(b,c)
    return qc

In [ ]:
def oracle():
    qc = QuantumCircuit(10)
    #Check edge 01, store in qubit 4
    qc += ccheck(10,0,1,4)
    #Check edge 02, store in qubit 5
    qc += ccheck(10,0,2,5)
    #Check edge 03, store in qubit 6
    qc += ccheck(10,0,3,6)

    qc.barrier()

    #add qubit 4
    qc.cx(4,7)

    #add qubit 5
    qc.ccx(5,7,8)
    qc.cx(5,7)
    
    #add qubit 6
    qc.ccx(6,7,8)
    qc.cx(6,7)

    #Check if qubits 7 and 8 store 11 and save this in qubit 9 
    qc.ccx(7,8,9)
    
    return qc

In [ ]:
def inversion_about_average():
    """Apply inversion about the average step of Grover's algorithm."""
    circuit = QuantumCircuit(10)
    for i in range(4):
        circuit.h(i)
    for i in range(4):
        circuit.x(i)
    circuit.h(3)
    circuit.mct([0,1,2],3)
    circuit.h(3)
    for i in range(4):
        circuit.x(i)
    for i in range(4):
        circuit.h(i)
    return circuit

In [ ]:
groverCircuit = QuantumCircuit(10,4)
for i in range(4):
    groverCircuit.h(i)

iterations = 2
for i in range(iterations):
    #query phase
    groverCircuit += oracle()
    #note that we store the result in qubit 9
    groverCircuit.z(9)
    groverCircuit += oracle().inverse()

    groverCircuit.barrier()

    #inversion phase
    groverCircuit += inversion_about_average()

#measure in reverse order
for i in range(4):    
    groverCircuit.measure(i,3-i)

backend = BasicAer.get_backend('qasm_simulator')
shots = 1024
results = execute(groverCircuit, backend=backend, shots=shots).result()
answer = results.get_counts()
print(answer)

{'1000': 493, '0111': 481, '1111': 4, '0100': 6, '1100': 4, '1011': 6, '0101': 1, '0110': 5, '1101': 2, '1010': 4, '0001': 4, '0011': 4, '0010': 1, '0000': 3, '1110': 5, '1001': 1}
